# SQL

## Ссылки

Основной синтаксис в конце [лекции](https://docs.google.com/presentation/d/14g4OHrgpVYGG7f5XZX-E0PBdptOFCCbAGDfq3IZIqdY/edit?usp=share_link)

https://dbfiddle.uk/ -- одна из онлайн сред, в которой можно работать

https://www.mysqltutorial.org/install-mysql/ -- тутор по установке MySQL сервера и клиента на вашей машине

https://www.mysqltutorial.org/mysql-basics/ -- хороший разбор базовых операций с SQL

Возможные среды для локальной работы с MySQL:
- [MySQL WorkBench](https://www.mysql.com/products/workbench/)
- [DBeaver](https://dbeaver.io/)
- [DataGrip](https://www.jetbrains.com/ru-ru/datagrip/)
- Плагин для VSCode

*Для вашей домашней работы вам не потребуется делать коннект к базе из Python, достаточно будет просто работать в отдельной среде*

Однако, если вам интересно разобраться, посмотрите в сторону подключения к базе с помощью pyODBC или mys Когда все будет настроено, запуск запроса к базе данных из Python будет выглядеть следующим образом:

```Python
import pyodbc # or import mysql
import pandas as pd

con = pyodbc.connect(...)  # передаются аргументы для подключения к вашей базе данных
# con = mysql.connector.connect(...)


sql = """
    ...
"""  # ваш запрос

res = pd.read_sql(sql, con)  # запрос в формате строки -- первый аргумент, объект коннекшена -- второй 
```

Некоторые ссылки, которые помогут разобраться:
- https://www.geeksforgeeks.org/python-mysql/
- https://dev.mysql.com/doc/connector-python/en/connector-python-introduction.html
- https://learn.microsoft.com/en-us/sql/connect/python/pyodbc/python-sql-driver-pyodbc?view=sql-server-ver16 -- только понадобится другой драйвер (для mySQL, а не MicrosoftSQL)
- https://youtu.be/eDXX5evRgQw

## Поехали

### Создадим базу и таблицу в ней

Создадим свою базу данных, добавим туда таблицу и наполним ее данными

```sql
show databases;  # смотрим на все доступные базы данных

create database knad_222_ftw;  # создаем свою бд

```

![](img/sql_example_1.png)

```sql

use knad_222_ftw;  # используем нашу базу данных

create table participants (
    student_id int(10) not null primary key, -- можно дописать auto_increment, тогда индекс будет увеличиваться автоматически
    name varchar(30) not null,
    age int
);


insert into participants(student_id, name, age)  -- добавляем один ряд в таблицу
    values (1, 'habilis', 42);
```

![](img/sql_example_2.png)

```sql

insert into participants(student_id, name, age)
    values(2, Kirill, 1337)
    values(3, Denis, 25);
    -- values(...)  можно добавлять много 

select * from participants;

drop table (if exists) participants;  -- удаляем таблицу. Если таблицы уже не существует и не стоит условия if exists, будет ошибка
```

![](img/sql_example_3.png)

### Sakila

Поработаем с датасетом [Sakila](https://dev.mysql.com/doc/sakila/en/sakila-structure-tables.html), описывающим данные сервиса проката фильмов

*В dffiddle.uk выбрать MySQL, и в верхнем селекторе вместо "No sample DB" выбрать "Sakila"*

Template почти любого запроса Select:

```sql
SELECT col1, SUM(col2) as col2sum, AVG(col3) as col3avg 
FROM table_name 
WHERE col4 = 'some_value' 
GROUP BY col1
(HAVING ...)
ORDER BY col2sum DESC
LIMIT 10;
```

Сохраненная сессия с семинара -- https://dbfiddle.uk/jS1lGrO4

1. Посмотрим, какие у нас есть таблицы

```sql
show databases;
use sakila;
show tables;
```

2. Какие актеры у нас есть в базе. Применим к ним условие.

```sql

select * from actor
limit 10;

select first_name, last_name
from actor
where first_name = 'Scarlett';
```

3. Сколько различных имен у актеров?

```sql
select count(distinct first_name) from actor;
```

4. Сколько каждого имени?

```sql
select first_name, count(*) as name_appearences
from actor 
group by first_name;
```

5. Какая средняя, минимальная и максимальные длины у фильмов в зависимости от их рейтинга и года выхода? Сохраним агрегат в таблицу

```sql
create table ratings_lengths as (
select rating, release_year, avg(length) as avg_len, min(length) as min_len, max(length) as max_len
from film
group by rating, release_year
);

select * from ratings_length;

```

6. Отфильтруем данные выше по некоторому условию на среднюю длину фильма.

```sql

select * from ratings_length
where avg_len < 115;

-- ИЛИ без создания новой таблицы. Используем вложенный select

select rating
, release_year
, avg_len
, min_len
, max_len
from (
    select rating
    , release_year
    , avg(length) as avg_len
    , min(length) as min_len
    , max(length) as max_len
    from film
    group by rating, release_year
) as temp
where temp.avg_len < 115;
```


7. Посмотрим на страны и города в нашей базе. Добавим колонку с названием страны к таблице с городами.

```sql

select * from city
limit 5;

select * from country
limit 5;

select a.city_id, a.city, a.country_id, b.country
from city a
left join country b on a.country_id = b.country_id
order by country
limit 50;

-- если названия соответвтующих колонок совпадают, можно писать using(column_names) вместо on...

```


Почитайте для понимания разных видов join:

- https://www.w3schools.com/mysql/mysql_join.asp 

- https://www.mysqltutorial.org/mysql-join/

![](http://www.securesolutions.no/wp-content/uploads/2014/07/joins-1.jpg)

8. Посмотрим на то, каким количеством городов представлены разные страны.

```sql
select a.country_id, b.country, count(distinct city) as num_cities
from city a
left join country b on a.country_id = b.country_id
group by country_id, country
order by num_cities desc;
```

9. Сделаем версию запроса выше с фильтрацией данных, где городов меньше 7. Сделаем двумя способами, с помощью [Having](https://www.mysqltutorial.org/mysql-having.aspx) и вложенного запроса

```sql

select a.country_id, b.country, count(distinct city) as num_cities
from city a
left join country b on a.country_id = b.country_id
group by country_id, country
having count(distinct city) > 7  -- или having num_cities > 7 
order by num_cities desc;

select t.*
from (
    select a.country_id, b.country, count(distinct city) as num_cities
    from city a
    left join country b on a.country_id = b.country_id
    group by country_id, country
) t
where t.num_cities > 7
order by t.num_cities desc;

```

Что еще можно посмотреть в базе для практики:

    - Сколько денег потратили пользователи на аренду фильмов всего? В разрезе по месяцам?
    - Из каких стран у нас больше вего покупателей?
    - Жители каких городов в среднем тратили больше всего денег?
    

И еще несколько примеров запросов с решениями ([source](https://datamastery.gitlab.io/exercises/sakila-queries.html)):
    
```sql
--  Which actors have the first name 'Scarlett'

select * from actor where first_name = 'Scarlett';

--  Which actors have the last name 'Johansson'

select * from actor where last_name like 'Johansson';

--  How many distinct actors last names are there?

select count(distinct last_name) from actor;

--  Which last names are not repeated?

select last_name from actor group by last_name having count(*) = 1;

--  Which last names appear more than once?

select last_name from actor group by last_name having count(*) > 1;

--  Which actor has appeared in the most films?

select actor.actor_id, actor.first_name, actor.last_name,
       count(actor_id) as film_count
from actor join film_actor using (actor_id)
group by actor_id
order by film_count desc
limit 1;

--  Is 'Academy Dinosaur' available for rent from Store 1?

--  Step 1: which copies are at Store 1?

select film.film_id, film.title, store.store_id, inventory.inventory_id
from inventory join store using (store_id) join film using (film_id)
where film.title = 'Academy Dinosaur' and store.store_id = 1;

--  Step 2: pick an inventory_id to rent:

select inventory.inventory_id
from inventory join store using (store_id)
     join film using (film_id)
     join rental using (inventory_id)
where film.title = 'Academy Dinosaur'
      and store.store_id = 1
      and not exists (select * from rental
                      where rental.inventory_id = inventory.inventory_id
                      and rental.return_date is null);

--  Insert a record to represent Mary Smith renting 'Academy Dinosaur' from Mike Hillyer at Store 1 today .

insert into rental (rental_date, inventory_id, customer_id, staff_id)
values (NOW(), 1, 1, 1);

--  When is 'Academy Dinosaur' due?

--  Step 1: what is the rental duration?

select rental_duration from film where film_id = 1;

--  Step 2: Which rental are we referring to -- the last one.

select rental_id from rental order by rental_id desc limit 1;

--  Step 3: add the rental duration to the rental date.

select rental_date,
       rental_date + interval
                   (select rental_duration from film where film_id = 1) day
                   as due_date
from rental
where rental_id = (select rental_id from rental order by rental_id desc limit 1);

--  What is that average length of all the films in the sakila DB?

select avg(length) from film;

--  What is the average length of films by category?

select category.name, avg(length)
from film join film_category using (film_id) join category using (category_id)
group by category.name
order by avg(length) desc;

-- Which film categories are long?

select category.name, avg(length)
from film join film_category using (film_id) join category using (category_id)
group by category.name
having avg(length) > (select avg(length) from film)
order by avg(length) desc;

--  Why does this query return the empty set?

select * from film natural join inventory;

```